# `nanoGPT`

## Install / Setup

### First Time Running

We need to install `ngpt` and setup the Shakespeare dataset

This will need to be ran the first time you are running this notebook.

Following the

```python
!python3 -m pip install nanoGPT
```

you will need to restart your runtime (Runtime -> Restart runtime)

After this, you should be able to

```python
>>> import ngpt
>>> ngpt.__file__
'/content/nanoGPT/src/ngpt/__init__.py'
```

In [1]:
%%bash

[ $(python3 -c 'import ngpt; print(ngpt.__file__)') ] && export HAS_NGPT=1 || export HAS_NGPT=0
[ $HAS_NGPT ] || git clone https://github.com/saforem2/nanoGPT;
[ $HAS_NGPT ] || python3 nanoGPT/data/shakespeare_char/prepare.py
[ $HAS_NGPT ] || python3 -m pip install -e nanoGPT -vvv

## Post Install

If installed correctly, you should be able to:

```python
>>> import ngpt
>>> ngpt.__file__
'/path/to/nanoGPT/src/ngpt/__init__.py'
```

In [2]:
%load_ext autoreload
%autoreload 2

import ngpt
from rich import print
print(ngpt.__file__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/src/ngpt/__init__.py

## Build Trainer

Explicitly, we:

1. `setup_torch(...)`
2. Build `cfg: DictConfig = get_config(...)`
3. Instnatiate `config: ExperimentConfig = instantiate(cfg)`
4. Build `trainer = Trainer(config)`

In [3]:
import os
from ezpz import setup_torch
from hydra.utils import instantiate
from ngpt.configs import get_config
from ngpt.trainer import Trainer

os.environ['MASTER_PORT'] = '5432'
rank = setup_torch('DDP', seed=1234)
cfg = get_config(
    [
        'data=shakespeare',
        'model=shakespeare',
        'optimizer=shakespeare',
        'train=shakespeare',
        'train.dtype=bfloat16',
        'train.max_iters=10000',
        'train.log_interval=250',
        'train.eval_interval=2000',
    ]
)
config = instantiate(cfg)
trainer = Trainer(config)

--------------------------------------------------------------------------

  Local host:   thetagpu24
  Local device: mlx5_0
--------------------------------------------------------------------------
RANK: 0 / 0


[2023-11-10 14:52:44][INFO][configs.py:264] - Rescaling GAS -> GAS // WORLD_SIZE = 1 // 1
[2023-11-10 14:52:44][INFO][configs.py:399] - Tokens per iteration: 16,384
[2023-11-10 14:52:44][INFO][configs.py:431] - Using <torch.amp.autocast_mode.autocast object at 0x7f0a5c3fbb20>
[2023-11-10 14:52:44][INFO][configs.py:437] - Initializing a new model from scratch
[2023-11-10 14:52:44][INFO][trainer.py:176] - Initializing a new model from scratch
[2023-11-10 14:52:45][INFO][model.py:160] - number of parameters: 10.65M
[2023-11-10 14:52:46][INFO][model.py:290] - num decayed parameter tensors: 26, with 10,740,096 parameters
[2023-11-10 14:52:46][INFO][model.py:291] - num non-decayed parameter tensors: 13, with 4,992 parameters


2023-11-10 14:52:47.233710: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[2023-11-10 14:52:50][INFO][model.py:297] - using fused AdamW: True


## Train Model

Legend:

|  name  |       description            |
|:------:|:----------------------------:|
| `step` | Current training step        |
| `loss` | Loss value                   |
| `dt`   | Time per step (in **ms**)    |
| `sps`  | Samples per second           |
| `mtps` | (million) Tokens per sec     |
| `mfu`  | Model Flops utilization[^1]  |

[^1]: in nits of A100 `bfloat16` peak FLOPS

In [4]:
trainer.train()

  0%|          | 0/10000 [00:00<?, ?it/s]

[2023-11-10 14:53:04][INFO][trainer.py:540] - step=250 loss=2.077 dt=26.609 sps=37.581 mtps=0.616 mfu=14.004 train_loss=4.299 val_loss=4.291
[2023-11-10 14:53:11][INFO][trainer.py:540] - step=500 loss=1.596 dt=26.093 sps=38.324 mtps=0.628 mfu=14.031 train_loss=4.299 val_loss=4.291
[2023-11-10 14:53:17][INFO][trainer.py:540] - step=750 loss=1.418 dt=26.343 sps=37.961 mtps=0.622 mfu=14.043 train_loss=4.299 val_loss=4.291
[2023-11-10 14:53:24][INFO][trainer.py:540] - step=1000 loss=1.332 dt=25.772 sps=38.801 mtps=0.636 mfu=14.084 train_loss=4.299 val_loss=4.291
[2023-11-10 14:53:31][INFO][trainer.py:540] - step=1250 loss=1.277 dt=25.746 sps=38.840 mtps=0.636 mfu=14.123 train_loss=4.299 val_loss=4.291
[2023-11-10 14:53:37][INFO][trainer.py:540] - step=1500 loss=1.234 dt=25.915 sps=38.588 mtps=0.632 mfu=14.149 train_loss=4.299 val_loss=4.291
[2023-11-10 14:53:44][INFO][trainer.py:540] - step=1750 loss=1.175 dt=25.756 sps=38.826 mtps=0.636 mfu=14.180 train_loss=4.299 val_loss=4.291
[2023-11-

## Evaluate Model

In [5]:
query = "what is the meaning of life?"
outputs = trainer.evaluate(query, num_samples=1, display=False)

In [6]:
from enrich.console import get_console
console = get_console()

console.print(outputs['0']['prompt'])
console.print(outputs['0']['formatted'])

[prompt]: 'what is the meaning of life?'

> 
> DUKE OF AUMERLE:
> Right gracious lord, I here protest,
> To make poor my good lord girl signs of Carlisle.
> 
> KING RICHARD II:
> Cousin, seize Hereford, and thou didst kill him;
> He hath resign'd his native resigns.
> I am no longer for than he is:
> Speak but for a little while a day,
> To whom I encounter: I have done what.
> 
> KING RICHARD II:
> Madam, have all the measure I enjoy'd
> The king is my father's temper:
> If you could proceed; then am I not broad
> Not inch to take it with age and to pluck away.
> 
> QUEEN